# Sample Usage

In [ ]:
from utils import convert_to_json
from metric.evaluator import get_evaluator

task = 'dialogue'

# a list of dialogue histories
src_list = ['hi , do you know much about the internet ? \n i know a lot about different sites and some website design , how about you ? \n\n']
# a list of additional context that should be included into the generated response
context_list = ['the 3 horizontal line menu on apps and websites is called a hamburger button .\n']
# a list of model outputs to be evaluated
output_list = ['i do too . did you know the 3 horizontal line menu on apps and websites is called the hamburger button ?']

# Prepare data for pre-trained evaluators
data = convert_to_json(output_list=output_list, 
                       src_list=src_list, context_list=context_list)
# Initialize evaluator for a specific task
evaluator = get_evaluator(task)
# Get multi-dimensional evaluation scores
eval_scores = evaluator.evaluate(data, print_result=True)

100%|██████████| 1/1 [00:00<00:00, 51.56it/s]


Evaluation scores are shown below:
+-------------------+----------+
|     Dimensions    |  Score   |
+-------------------+----------+
|    naturalness    | 0.950217 |
|     coherence     | 0.973135 |
|    engagingness   | 1.750486 |
|    groundedness   | 0.999566 |
| understandability | 0.946209 |
|      overall      | 1.123923 |
+-------------------+----------+


In [6]:
from utils import convert_to_json
from metric.evaluator import get_evaluator

task = 'dialogue'

# Batch inputs: multiple dialogue histories, contexts, and model outputs
src_list = [
    'hi , do you know much about the internet ? \n i know a lot about different sites and some website design , how about you ? \n\n',
    'what is your favorite color ? \n i like blue a lot , but sometimes i prefer green . \n\n'
]
context_list = [
    'the 3 horizontal line menu on apps and websites is called a hamburger button .\n',
    'colors can reflect your mood and personality .\n'
]
output_list = [
    'i do too . did you know the 3 horizontal line menu on apps and websites is called the hamburger button ?',
    'i like blue as well . it is calming and reminds me of the ocean .'
]

# Prepare data for pre-trained evaluators
data = convert_to_json(output_list=output_list, 
                       src_list=src_list, context_list=context_list)

# Initialize evaluator for a specific task
evaluator = get_evaluator(task)

# Get multi-dimensional evaluation scores for the batch
eval_scores = evaluator.evaluate(data, print_result=False)

# Display scores
for i, score in enumerate(eval_scores):
    print(f"Scores for example {i + 1}: {score}")


100%|██████████| 1/1 [00:00<00:00, 50.35it/s]

Scores for example 1: {'naturalness': 0.9502174201360719, 'coherence': 0.9731347836152868, 'engagingness': 1.7504860805525295, 'groundedness': 0.9995656267195939, 'understandability': 0.9462095037239142, 'overall': 1.1239226829494793}
Scores for example 2: {'naturalness': 0.9675946477090701, 'coherence': 0.998674558536015, 'engagingness': 1.9936029005678884, 'groundedness': 0.9857853472625128, 'understandability': 0.9632170362278318, 'overall': 1.1817748980606635}


In [7]:
eval_scores

[{'naturalness': 0.9502174201360719,
  'coherence': 0.9731347836152868,
  'engagingness': 1.7504860805525295,
  'groundedness': 0.9995656267195939,
  'understandability': 0.9462095037239142,
  'overall': 1.1239226829494793},
 {'naturalness': 0.9675946477090701,
  'coherence': 0.998674558536015,
  'engagingness': 1.9936029005678884,
  'groundedness': 0.9857853472625128,
  'understandability': 0.9632170362278318,
  'overall': 1.1817748980606635}]

# Response Generation Evaluation

In [18]:
import pandas as pd
import warnings
import logging

warnings.filterwarnings("ignore")
logging.getLogger('transformers').setLevel(logging.ERROR)

# Set the logging level to ERROR to ignore warnings
logging.getLogger("transformers").setLevel(logging.ERROR)


In [19]:
Dataset = "Blended Skill Talk"                                # Synthetic-PersonaChat, Blended Skill Talk, PEC, ConvAI2, FoCus, IT-ConvAI2
LLM_name = "gpt-4o-mini"                                # Mistral-7B-Instruct, Llama3-1-8B-Instruct, Qwen2-7B-Instruct,  gpt-3.5-turbo, gpt-4-turbo, gpt-4o-mini
COT_SETUP = False

In [20]:
df = pd.read_csv(f'../Prompts/{Dataset}.csv')
print("Shape:", df.shape)

df.head()

Shape: (980, 3)


,personas,act_response,context
0,[User 1 persona]: ['i hate talking to people.'...,"I think it's because in my head, I think every...","User1: Wow, I am never shy. Do you have anxiet..."
1,[User 1 persona]: ['i have three daughters.' '...,What does your turtle eat? Is it hard to take...,User1: My turtle ran away from me today.\nUser...
2,[User 1 persona]: ['i hate the taste of fish.'...,"Yeah, kids grow up so quickly",User1: Our son in the Army is taking a leave t...
3,[User 1 persona]: ['my favorite movie is good ...,"Wow, you've done a marathon? I run a bit, but...","User1: that's awesome , i like running in the ..."
4,[User 1 persona]: ['my hair is black.' 'i like...,I would suggest a fitness place with a rock wa...,User1: Are there different skill levels? \nUse...


In [21]:
Dataset

'Blended Skill Talk'

In [22]:
# ### Only For: Blended Skill Talk
if Dataset == "Blended Skill Talk":
    df['personas'] = df['personas'].str.replace(r'\[User 1 persona\]:|\[|\]|"|\'', '', regex=True).str.strip()

# ### Only For: PEC
if Dataset == "PEC":
    df['personas'] = df['personas'].str.replace(r'\[Responder persona\]:|\[|\]|"|\'', '', regex=True).str.strip()


print(df.isnull().sum())
df.head(6)

personas        0
act_response    0
context         0
dtype: int64


,personas,act_response,context
0,i hate talking to people. i believe dragons ar...,"I think it's because in my head, I think every...","User1: Wow, I am never shy. Do you have anxiet..."
1,i have three daughters. my wife and i like to ...,What does your turtle eat? Is it hard to take...,User1: My turtle ran away from me today.\nUser...
2,i hate the taste of fish. i like to paint.,"Yeah, kids grow up so quickly",User1: Our son in the Army is taking a leave t...
3,my favorite movie is good burger. i like canni...,"Wow, you've done a marathon? I run a bit, but...","User1: that's awesome , i like running in the ..."
4,my hair is black. i like rock climbing.,I would suggest a fitness place with a rock wa...,User1: Are there different skill levels? \nUse...
5,my dad works at the mill and my mom is a teach...,"I'm sure you'll do great. In second grade, tha...",User1: This is the first time I drop my kids o...


In [23]:
COT_ = "-COT" if COT_SETUP else ""
 
response = pd.read_csv(f'../Responses/{Dataset}/{LLM_name}{COT_}.csv')
print("Shape:", response.shape)

print("\nMissing Values:")
print(response.isnull().sum())

response

Shape: (980, 2)

Missing Values:
gen_response     1
response_time    0
dtype: int64


,gen_response,response_time
0,I guess it's just the pressure of talking to p...,1.002316
1,That's hilarious! I can just imagine your turt...,0.831460
2,"I know, right? It feels like just yesterday he...",0.902882
3,That's a smart idea! Canning can really help w...,0.789975
4,"To start rock climbing, you might want to find...",2.361459
...,...,...
975,That's great! Mind maps can be really helpful ...,0.816989
976,I totally understand the dilemma you're facing...,1.033905
977,I totally get that! Blue is such a beautiful c...,0.813419
978,It sounds like you're going through a really t...,1.067153


In [24]:
# Calculate maximum number of words in each column
max_response_length = response['gen_response'].dropna().apply(lambda x: len(x.split())).max()

print(f"Maximum Response Length (in words): {max_response_length}")

Maximum Response Length (in words): 82


In [25]:
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Initialize stop words
stop_words = set(stopwords.words('english'))

# Function to preprocess text
def preprocess_text(text, remove_stop_words=True):
    if pd.isnull(text):
        return None
    text = text.lower()  # Lowercasing
    text = text.translate(str.maketrans('', '', string.punctuation))  # Removing punctuation
    tokens = word_tokenize(text)  # Tokenization
    if remove_stop_words:
        tokens = [word for word in tokens if word not in stop_words]  # Removing stop words
    return ' '.join(tokens)  # Join tokens back into a single string

# Create eval_df
eval_df = pd.DataFrame({
    'personas': df['personas'],
    'context': df['context'],
    'gen_response': response['gen_response'],
    'response_time': response['response_time']
})

print(eval_df.isnull().sum())
eval_df.head()

personas         0
context          0
gen_response     1
response_time    0
dtype: int64


,personas,context,gen_response,response_time
0,i hate talking to people. i believe dragons ar...,"User1: Wow, I am never shy. Do you have anxiet...",I guess it's just the pressure of talking to p...,1.002316
1,i have three daughters. my wife and i like to ...,User1: My turtle ran away from me today.\nUser...,That's hilarious! I can just imagine your turt...,0.831460
2,i hate the taste of fish. i like to paint.,User1: Our son in the Army is taking a leave t...,"I know, right? It feels like just yesterday he...",0.902882
3,my favorite movie is good burger. i like canni...,"User1: that's awesome , i like running in the ...",That's a smart idea! Canning can really help w...,0.789975
4,my hair is black. i like rock climbing.,User1: Are there different skill levels? \nUse...,"To start rock climbing, you might want to find...",2.361459


In [26]:
import torch
device = 0 if torch.cuda.is_available() else -1  # device set to 0 for GPU, -1 for CPU
# device = -1

In [27]:
device

0

**Finalized Input Mapping:**

- src_list: Use the context column (conversation history).
- context_list: Use the flattened and cleaned persona column.
- output_list: Use the gen_response (the response your model generates).

**Note:**

The act_response (true or reference response) is not required as an input for the UniEval evaluation process because UniEval evaluates the generated response (gen_response) based on how well it fits the provided context (conversation history) and additional persona information. 

In [28]:
from utils import convert_to_json
from metric.evaluator import get_evaluator

def calculate_unieval_scores(personas, contexts, gen_responses):
    """
    Calculates UniEval scores for a batch of inputs.

    Args:
        personas (list): List of persona information as additional context.
        contexts (list): List of conversation histories leading to the responses.
        gen_responses (list): List of generated responses to be evaluated.

    Returns:
        list: A list of dictionaries containing UniEval scores for each input.
    """
    # Flatten personas if they are lists
    personas = [' '.join(p) if isinstance(p, list) else p for p in personas]

    # Prepare inputs for UniEval
    data = convert_to_json(output_list=gen_responses, src_list=contexts, context_list=personas)

    # Initialize the evaluator for dialogue tasks
    evaluator = get_evaluator('dialogue')

    # Evaluate and obtain scores for all inputs
    eval_scores = evaluator.evaluate(data, print_result=False)

    return eval_scores


In [29]:
# Define the worst UniEval score as a dictionary
worst_unieval_score = {
    'naturalness': 0.0,
    'coherence': 0.0,
    'engagingness': 0.0,
    'groundedness': 0.0,
    'understandability': 0.0,
    'overall': 0.0
}


In [30]:
from tqdm import tqdm
import pandas as pd

# Function to evaluate in batches or the entire DataFrame
batch_size = 200  # Adjust batch size as needed

# List to store all UniEval scores
all_unieval_scores = []

# Split into batches if necessary
for i in tqdm(range(0, len(eval_df), batch_size), desc="Evaluating batches"):
    batch = eval_df.iloc[i:i+batch_size]

    # Extract relevant fields from the batch
    personas = batch['personas'].tolist()
    contexts = batch['context'].tolist()
    gen_responses = batch['gen_response'].tolist()

    # Check for NaN responses and handle them
    valid_indices = [j for j, response in enumerate(gen_responses) if pd.notna(response) and response.strip() != '']
    invalid_indices = [j for j, response in enumerate(gen_responses) if j not in valid_indices]

    # Prepare valid inputs
    valid_personas = [personas[j] for j in valid_indices]
    valid_contexts = [contexts[j] for j in valid_indices]
    valid_gen_responses = [gen_responses[j] for j in valid_indices]

    # Evaluate valid inputs
    if valid_personas:
        eval_scores = calculate_unieval_scores(valid_personas, valid_contexts, valid_gen_responses)
        all_unieval_scores.extend(eval_scores)

    # Append worst scores for invalid inputs
    all_unieval_scores.extend([worst_unieval_score] * len(invalid_indices))

# Convert all scores into a DataFrame
metrics_df = pd.DataFrame(all_unieval_scores)

# Rename columns for clarity
metrics_df.columns = [
    "UniEval Naturalness",
    "UniEval Coherence",
    "UniEval Engagingness",
    "UniEval Groundedness",
    "UniEval Understandability",
    "UniEval Overall"
]

# Combine with original DataFrame if needed
eval_df = pd.concat([eval_df.reset_index(drop=True), metrics_df.reset_index(drop=True)], axis=1)

metrics_df

Evaluating batches:   0%|          | 0/5 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Evaluating batches:  20%|██        | 1/5 [01:28<05:53, 88.26s/it]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Evaluating batches:  40%|████      | 2/5 [03:01<04:32, 90.96s/it]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this war

,UniEval Naturalness,UniEval Coherence,UniEval Engagingness,UniEval Groundedness,UniEval Understandability,UniEval Overall
0,0.946804,0.971796,3.004960,0.998948,0.939764,1.372455
1,0.952231,0.997851,3.080799,0.010788,0.946839,1.197702
2,0.971357,0.998264,2.206288,0.907607,0.968676,1.210438
3,0.956967,0.998880,2.881239,0.983022,0.954477,1.354917
4,0.859593,0.999080,2.219915,0.990899,0.901610,1.194219
...,...,...,...,...,...,...
975,0.962115,0.996908,1.006736,0.280973,0.957939,0.840934
976,0.959471,0.994535,3.864800,0.972928,0.957468,1.549840
977,0.959926,0.999218,2.005206,0.002897,0.953072,0.984064
978,0.963703,0.999613,3.864537,0.913209,0.960440,1.540300


In [31]:
# Calculate the mean (average) and standard deviation, rounded to 2 decimal places
avg_values = metrics_df.mean().round(2)
std_values = metrics_df.std(ddof=0).round(2)  # Use ddof=0 for population standard deviation

# Combine the average and standard deviation into the format "avg ± std"
combined_values = avg_values.astype(str) + " ± " + std_values.astype(str)

# Insert the LLM name at the beginning of the combined values
combined_values = combined_values.tolist()
combined_values.insert(0, LLM_name)

In [32]:
# Create a DataFrame for the combined average ± std row
result_df = pd.DataFrame([combined_values], columns=['Model'] + metrics_df.columns.tolist())
result_df

,Model,UniEval Naturalness,UniEval Coherence,UniEval Engagingness,UniEval Groundedness,UniEval Understandability,UniEval Overall
0,gpt-4o-mini,0.96 ± 0.03,0.99 ± 0.03,2.9 ± 0.84,0.57 ± 0.41,0.96 ± 0.03,1.27 ± 0.19


In [33]:
import pandas as pd

# Load the existing Excel file and update or append the average row
output_path = f'../Evaluations/{Dataset}{COT_}-results.xlsx'

try:
    # Load existing data
    existing_df = pd.read_excel(output_path)
    
    # Check if the model name already exists
    if LLM_name in existing_df['Model'].values:
        # Update the row by appending the new columns
        existing_index = existing_df.loc[existing_df['Model'] == LLM_name].index[0]
        for col in result_df.columns:
            if col not in existing_df.columns:
                existing_df[col] = None  # Add new column if missing
            existing_df.at[existing_index, col] = result_df[col].values[0]  # Update column values
    else:
        # Append the new data
        existing_df = pd.concat([existing_df, result_df], ignore_index=True)
except FileNotFoundError:
    # If the file does not exist, create a new DataFrame
    existing_df = result_df

# # Save the updated DataFrame to an Excel file
existing_df.to_excel(output_path, index=False)

existing_df


,Model,P Consistency Score,C Score,UE Score,BLEU,R1,R2,RL,METEOR,BERTScore_Prec,...,`,Persona Distance,response_time,Failure Ratio,UniEval Naturalness,UniEval Coherence,UniEval Engagingness,UniEval Groundedness,UniEval Understandability,UniEval Overall
0,Llama3-1-8B-Instruct,0.79 ± 0.41,-0.11 ± 0.55,0.33 ± 0.7,0.01 ± 0.02,0.12 ± 0.08,0.02 ± 0.04,0.11 ± 0.08,0.12 ± 0.09,0.81 ± 0.17,...,0.06 ± 0.1,0.45 ± 0.17,4.28 ± 0.26,0.043 ± 0.00,0.92 ± 0.19,0.92 ± 0.25,2.42 ± 1.12,0.45 ± 0.43,0.92 ± 0.19,1.12 ± 0.32
1,Mistral-7B-Instruct,0.81 ± 0.39\t,-0.0 ± 0.62,0.48 ± 0.79\t,0.01 ± 0.01,0.1 ± 0.08,0.01 ± 0.03,0.1 ± 0.07,0.11 ± 0.08,0.8 ± 0.19,...,0.08 ± 0.1,0.46 ± 0.18,3.87 ± 0.72,0.052 ± 0.00,0.9 ± 0.21,0.93 ± 0.23,2.55 ± 1.18,0.54 ± 0.44,0.89 ± 0.21,1.16 ± 0.35
2,Qwen2-7B-Instruct,0.79 ± 0.41\t,-0.11 ± 0.56,0.39 ± 0.75,0.01 ± 0.01,0.11 ± 0.07,0.01 ± 0.03,0.1 ± 0.07,0.12 ± 0.08,0.82 ± 0.13,...,0.05 ± 0.08,0.46 ± 0.15,2.21 ± 0.67,0.024 ± 0.00,0.93 ± 0.15,0.97 ± 0.15,2.98 ± 1.18,0.44 ± 0.44,0.93 ± 0.15,1.25 ± 0.31
3,gpt-3.5-turbo,0.83 ± 0.38,-0.01 ± 0.57\t,0.51 ± 0.82,0.01 ± 0.02,0.13 ± 0.09,0.02 ± 0.04,0.11 ± 0.08,0.13 ± 0.09,0.85 ± 0.02,...,0.07 ± 0.11,0.47 ± 0.15,0.91 ± 0.21,0.0 ± 0.00,0.96 ± 0.02,0.97 ± 0.13,2.33 ± 0.83,0.28 ± 0.4,0.95 ± 0.02,1.1 ± 0.18
4,gpt-4-turbo,0.86 ± 0.35\t,-0.05 ± 0.48,0.56 ± 0.84,0.01 ± 0.02,0.12 ± 0.08,0.02 ± 0.04,0.11 ± 0.07,0.12 ± 0.09,0.85 ± 0.02,...,0.06 ± 0.07,0.47 ± 0.14,1.84 ± 0.5,0.0 ± 0.00,0.96 ± 0.01,0.98 ± 0.05,2.43 ± 0.78,0.53 ± 0.42,0.96 ± 0.01,1.17 ± 0.18
5,gpt-4o-mini,0.84 ± 0.37\t,-0.1 ± 0.46,0.38 ± 0.74,0.01 ± 0.01,0.13 ± 0.08,0.02 ± 0.03,0.12 ± 0.07,0.14 ± 0.09,0.85 ± 0.03,...,0.05 ± 0.07,0.48 ± 0.13,0.94 ± 0.28,0.001 ± 0.00,0.96 ± 0.03,0.99 ± 0.03,2.9 ± 0.84,0.57 ± 0.41,0.96 ± 0.03,1.27 ± 0.19


In [34]:
response = pd.read_excel(f'../Evaluations/{Dataset}{COT_}-results.xlsx')
response

,Model,P Consistency Score,C Score,UE Score,BLEU,R1,R2,RL,METEOR,BERTScore_Prec,...,`,Persona Distance,response_time,Failure Ratio,UniEval Naturalness,UniEval Coherence,UniEval Engagingness,UniEval Groundedness,UniEval Understandability,UniEval Overall
0,Llama3-1-8B-Instruct,0.79 ± 0.41,-0.11 ± 0.55,0.33 ± 0.7,0.01 ± 0.02,0.12 ± 0.08,0.02 ± 0.04,0.11 ± 0.08,0.12 ± 0.09,0.81 ± 0.17,...,0.06 ± 0.1,0.45 ± 0.17,4.28 ± 0.26,0.043 ± 0.00,0.92 ± 0.19,0.92 ± 0.25,2.42 ± 1.12,0.45 ± 0.43,0.92 ± 0.19,1.12 ± 0.32
1,Mistral-7B-Instruct,0.81 ± 0.39\t,-0.0 ± 0.62,0.48 ± 0.79\t,0.01 ± 0.01,0.1 ± 0.08,0.01 ± 0.03,0.1 ± 0.07,0.11 ± 0.08,0.8 ± 0.19,...,0.08 ± 0.1,0.46 ± 0.18,3.87 ± 0.72,0.052 ± 0.00,0.9 ± 0.21,0.93 ± 0.23,2.55 ± 1.18,0.54 ± 0.44,0.89 ± 0.21,1.16 ± 0.35
2,Qwen2-7B-Instruct,0.79 ± 0.41\t,-0.11 ± 0.56,0.39 ± 0.75,0.01 ± 0.01,0.11 ± 0.07,0.01 ± 0.03,0.1 ± 0.07,0.12 ± 0.08,0.82 ± 0.13,...,0.05 ± 0.08,0.46 ± 0.15,2.21 ± 0.67,0.024 ± 0.00,0.93 ± 0.15,0.97 ± 0.15,2.98 ± 1.18,0.44 ± 0.44,0.93 ± 0.15,1.25 ± 0.31
3,gpt-3.5-turbo,0.83 ± 0.38,-0.01 ± 0.57\t,0.51 ± 0.82,0.01 ± 0.02,0.13 ± 0.09,0.02 ± 0.04,0.11 ± 0.08,0.13 ± 0.09,0.85 ± 0.02,...,0.07 ± 0.11,0.47 ± 0.15,0.91 ± 0.21,0.0 ± 0.00,0.96 ± 0.02,0.97 ± 0.13,2.33 ± 0.83,0.28 ± 0.4,0.95 ± 0.02,1.1 ± 0.18
4,gpt-4-turbo,0.86 ± 0.35\t,-0.05 ± 0.48,0.56 ± 0.84,0.01 ± 0.02,0.12 ± 0.08,0.02 ± 0.04,0.11 ± 0.07,0.12 ± 0.09,0.85 ± 0.02,...,0.06 ± 0.07,0.47 ± 0.14,1.84 ± 0.5,0.0 ± 0.00,0.96 ± 0.01,0.98 ± 0.05,2.43 ± 0.78,0.53 ± 0.42,0.96 ± 0.01,1.17 ± 0.18
5,gpt-4o-mini,0.84 ± 0.37\t,-0.1 ± 0.46,0.38 ± 0.74,0.01 ± 0.01,0.13 ± 0.08,0.02 ± 0.03,0.12 ± 0.07,0.14 ± 0.09,0.85 ± 0.03,...,0.05 ± 0.07,0.48 ± 0.13,0.94 ± 0.28,0.001 ± 0.00,0.96 ± 0.03,0.99 ± 0.03,2.9 ± 0.84,0.57 ± 0.41,0.96 ± 0.03,1.27 ± 0.19
